In [2]:
import os
import torch
import pandas as pd
from transformers import AutoTokenizer

/home/ec2-user/miniconda3/envs/capstone/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from vllm import LLM, SamplingParams

INFO 06-24 17:07:31 [__init__.py:244] Automatically detected platform cuda.


2025-06-24 17:07:34,200	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
os.environ["HF_TOKEN"] = ""

In [5]:
!nvidia-smi

Tue Jun 24 17:08:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.20             Driver Version: 570.133.20     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10G                    On  |   00000000:00:1E.0 Off |                    0 |
|  0%   33C    P8              9W /  300W |       0MiB /  23028MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Loading our LLM onto vLLM and preparing tokenizer

Selected LLM: [meta-llama/Llama-3.2-3B-Instruct](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct)

In [10]:
if torch.cuda.is_available():
    print("GPU is available!")
    print(f"Using device: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU. Using CPU.")

GPU is available!
Using device: NVIDIA A10G


In [11]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"

In [12]:
sampling_params = SamplingParams(temperature=0.1, top_p=1, max_tokens=512)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [13]:
llm = LLM(model=model_id, dtype=torch.bfloat16, max_model_len=25000) # one of the prompts has lots of tokens so

INFO 06-24 17:08:47 [config.py:823] This model supports multiple tasks: {'embed', 'generate', 'reward', 'classify', 'score'}. Defaulting to 'generate'.
INFO 06-24 17:08:47 [config.py:2195] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 06-24 17:08:49 [utils.py:2597] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
WARNING 06-24 17:08:50 [env_override.py:17] NCCL_CUMEM_ENABLE is set to 0, skipping override. This may increase memory overhead with cudagraph+allreduce: https://github.com/NVIDIA/nccl/issues/1234
INFO 06-24 17:08:53 [__init__.py:244] Automatically detected platform cuda.
INFO 06-24 17:08:56 [core.py:455] Waiting for init message from front-end.
INFO 06-24 17:08:56 [core.py:70] Initializing a V1 LLM engine (v0.9.1) with config: model='meta-llama/Llama-3.2-3B-Instruc

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  4.90it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  1.91it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  2.10it/s]



INFO 06-24 17:09:28 [default_loader.py:272] Loading weights took 1.19 seconds
INFO 06-24 17:09:28 [gpu_model_runner.py:1624] Model loading took 6.0165 GiB and 30.227260 seconds
INFO 06-24 17:09:34 [backends.py:462] Using cache directory: /home/ec2-user/.cache/vllm/torch_compile_cache/63818febe3/rank_0_0 for vLLM's torch.compile
INFO 06-24 17:09:34 [backends.py:472] Dynamo bytecode transform time: 5.78 s
INFO 06-24 17:09:38 [backends.py:161] Cache the graph of shape None for later use
INFO 06-24 17:10:00 [backends.py:173] Compiling a graph for general shape takes 25.63 s
INFO 06-24 17:10:08 [monitor.py:34] torch.compile takes 31.41 s in total
INFO 06-24 17:10:09 [gpu_worker.py:227] Available KV cache memory: 12.61 GiB
INFO 06-24 17:10:10 [kv_cache_utils.py:715] GPU KV cache size: 118,080 tokens
INFO 06-24 17:10:10 [kv_cache_utils.py:719] Maximum concurrency for 25,000 tokens per request: 4.72x
INFO 06-24 17:10:32 [gpu_model_runner.py:2048] Graph capturing finished in 22 secs, took 0.45 

# Testing on a single prompt for speed

In [14]:
system_prompt = '''You are an expert summarizer helping a user prepare input for a sentence embedding model with a strict 512-token input limit.
The user will always include the full **title** of the news article. Your job is to **rewrite or summarize the news article text only**, using no more than the available tokens provided. The goal is to preserve **all important meaning** from the article without exceeding the token budget.

You must:
- Leave the title unchanged
- Output **only the rewritten article text**
- Not include anything non-relevant stuff in your response

You will be provided Input details.
'''

prompt = '''Input details:

    Title: LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]

    Title Token Count: 35

    Original Text: No comment is expected from Barack Obama Members of the #FYF911 or #FukYoFlag and #BlackLivesMatter movements called for the lynching and hanging of white people and cops. They encouraged others on a radio show Tuesday night to  turn the tide  and kill white people and cops to send a message about the killing of black people in America.One of the F***YoFlag organizers is called  Sunshine.  She has a radio blog show hosted from Texas called,  Sunshine s F***ing Opinion Radio Show. A snapshot of her #FYF911 @LOLatWhiteFear Twitter page at 9:53 p.m. shows that she was urging supporters to  Call now!! #fyf911 tonight we continue to dismantle the illusion of white Below is a SNAPSHOT Twitter Radio Call Invite   #FYF911The radio show aired at 10:00 p.m. eastern standard time.During the show, callers clearly call for  lynching  and  killing  of white people.A 2:39 minute clip from the radio show can be heard here. It was provided to Breitbart Texas by someone who would like to be referred to as  Hannibal.  He has already received death threats as a result of interrupting #FYF911 conference calls.An unidentified black man said  when those mother f**kers are by themselves, that s when when we should start f***ing them up. Like they do us, when a bunch of them ni**ers takin  one of us out, that s how we should roll up.  He said,  Cause we already roll up in gangs anyway. There should be six or seven black mother f**ckers, see that white person, and then lynch their ass. Let s turn the tables. They conspired that if  cops started losing people,  then  there will be a state of emergency. He speculated that one of two things would happen,  a big-ass [R s?????] war,  or  ni**ers, they are going to start backin  up. We are already getting killed out here so what the f**k we got to lose? Sunshine could be heard saying,  Yep, that s true. That s so f**king true. He said,  We need to turn the tables on them. Our kids are getting shot out here. Somebody needs to become a sacrifice on their side.He said,  Everybody ain t down for that s**t, or whatever, but like I say, everybody has a different position of war.  He continued,  Because they don t give a f**k anyway.  He said again,  We might as well utilized them for that s**t and turn the tables on these n**ers. He said, that way  we can start lookin  like we ain t havin  that many casualties, and there can be more causalities on their side instead of ours. They are out their killing black people, black lives don t matter, that s what those mother f**kers   so we got to make it matter to them. Find a mother f**ker that is alone. Snap his ass, and then f***in hang him from a damn tree. Take a picture of it and then send it to the mother f**kers. We  just need one example,  and  then people will start watchin .  This will turn the tables on s**t, he said. He said this will start  a trickle-down effect.  He said that when one white person is hung and then they are just  flat-hanging,  that will start the  trickle-down effect.  He continued,  Black people are good at starting trends. He said that was how  to get the upper-hand. Another black man spoke up saying they needed to kill  cops that are killing us. The first black male said,  That will be the best method right there. Breitbart Texas previously reported how Sunshine was upset when  racist white people  infiltrated and disrupted one of her conference calls. She subsequently released the phone number of one of the infiltrators. The veteran immediately started receiving threatening calls.One of the #F***YoFlag movement supporters allegedly told a veteran who infiltrated their publicly posted conference call,  We are going to rape and gut your pregnant wife, and your f***ing piece of sh*t unborn creature will be hung from a tree. Breitbart Texas previously encountered Sunshine at a Sandra Bland protest at the Waller County Jail in Texas, where she said all white people should be killed. She told journalists and photographers,  You see this nappy-ass hair on my head?   That means I am one of those more militant Negroes.  She said she was at the protest because  these redneck mother-f**kers murdered Sandra Bland because she had nappy hair like me. #FYF911 black radicals say they will be holding the  imperial powers  that are actually responsible for the terrorist attacks on September 11th accountable on that day, as reported by Breitbart Texas. There are several websites and Twitter handles for the movement. Palmetto Star  describes himself as one of the head organizers. He said in a YouTube video that supporters will be burning their symbols of  the illusion of their superiority,  their  false white supremacy,  like the American flag, the British flag, police uniforms, and Ku Klux Klan hoods.Sierra McGrone or  Nocturnus Libertus  posted,  you too can help a young Afrikan clean their a** with the rag of oppression.  She posted two photos, one that appears to be herself, and a photo of a black man, wiping their naked butts with the American flag.For entire story: Breitbart News

    Original Text Token Count: 1200

    Max Allowed Total Tokens (Title + Text): 512

    Available Tokens for Text Summary: 477


    Now rewrite the article text only (not the title) within the available token budget. Your output should be clean, self-contained, and informative.
    Do not add any information that is not present in the original news text. Your output must be exactly text summary and nothing else'''

In [15]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt},
]

In [16]:
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [17]:
outputs = llm.generate([prompt]*100, sampling_params)

Processed prompts: 100%|██████████| 100/100 [00:10<00:00,  9.40it/s, est. speed input: 13907.32 toks/s, output: 2008.85 toks/s]


In [18]:
print(outputs[-1].outputs[0].text)

Members of the #FYF911 and #BlackLivesMatter movements called for the lynching and hanging of white people and cops in response to the killing of black people in America. A radio show host, Sunshine, urged supporters to kill white people and cops, saying "we should start f***ing them up" when they are alone. She suggested that if cops started losing people, there would be a state of emergency, leading to a big war or more black people rising up. Sunshine said, "We need to turn the tables on them. Our kids are getting shot out here. Somebody needs to become a sacrifice on their side." Another black man said, "Find a mother f**ker that is alone. Snap his ass, and then f***in hang him from a damn tree." He also suggested killing cops that are killing black people.


In [19]:
len(tokenizer.encode(outputs[-1].outputs[0].text, return_tensors='pt')[0])

174

Well, processing 100 prompts took ~11 seconds, and we have an average of 2500 output tokens per second which is not bad at all. Lets process all prompts now. 

# Reading file and preparing prompts

In [20]:
df = pd.read_csv("Summary_prompts_FinalTesting.csv")

In [21]:
print(df.shape)
df.head()

(19248, 2)


,text,label
0,Input details:\n\n Title: Donald Trump Sen...,1
1,Input details:\n\n Title: Sheriff David Cl...,1
2,Input details:\n\n Title: Trump Is So Obse...,1
3,Input details:\n\n Title: Former CIA Direc...,1
4,Input details:\n\n Title: Heiress To Disne...,1


In [22]:
print(df.text.values[0])

Input details:

    Title:  Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing

    Title Token Count: 17

    Original Text: Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as y

In [27]:
from tqdm import tqdm

In [28]:
df

,text,label,token_count,prompt
0,Input details:\n\n Title: Donald Trump Sen...,1,1109,<|begin_of_text|><|start_header_id|>system<|en...
1,Input details:\n\n Title: Sheriff David Cl...,1,1299,<|begin_of_text|><|start_header_id|>system<|en...
2,Input details:\n\n Title: Trump Is So Obse...,1,1050,<|begin_of_text|><|start_header_id|>system<|en...
3,Input details:\n\n Title: Former CIA Direc...,1,924,<|begin_of_text|><|start_header_id|>system<|en...
4,Input details:\n\n Title: Heiress To Disne...,1,989,<|begin_of_text|><|start_header_id|>system<|en...
...,...,...,...,...
19243,"Input details:\n\n Title: U.S., North Korea...",0,1237,<|begin_of_text|><|start_header_id|>system<|en...
19244,"Input details:\n\n Title: U.S., North Korea...",0,1237,<|begin_of_text|><|start_header_id|>system<|en...
19245,"Input details:\n\n Title: U.S., North Korea...",0,1160,<|begin_of_text|><|start_header_id|>system<|en...
19246,Input details:\n\n Title: North Korea shipm...,0,906,<|begin_of_text|><|start_header_id|>system<|en...


In [24]:
total_prompts = []
token_count = []

for current_prompt in tqdm(df.text.values):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": current_prompt},
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    total_prompts.append(prompt)
    encoded = tokenizer.encode(prompt, return_tensors='pt')
    token_count.append(len(encoded[0]))

100%|██████████| 19248/19248 [00:51<00:00, 374.20it/s]


In [25]:
df['token_count'] = token_count
df['prompt'] = total_prompts

In [26]:
df["token_count"].describe(), df["token_count"].quantile(0.999)

(count    19248.000000
 mean      1176.497039
 std        509.352368
 min        775.000000
 25%        922.000000
 50%       1048.000000
 75%       1266.000000
 max      10636.000000
 Name: token_count, dtype: float64,
 np.float64(7222.0))

In [29]:
df = df[df["token_count"] < 20000]

In [30]:
df.shape

(19248, 4)

In [31]:
total_prompts = df.prompt.values

# Summarization using vLLM + Llama-3.2-3B-Instruct

In [32]:
print(total_prompts[-10])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 24 Jun 2025

You are an expert summarizer helping a user prepare input for a sentence embedding model with a strict 512-token input limit.
The user will always include the full **title** of the news article. Your job is to **rewrite or summarize the news article text only**, using no more than the available tokens provided. The goal is to preserve **all important meaning** from the article without exceeding the token budget.

You must:
- Leave the title unchanged
- Output **only the rewritten article text**
- Not include anything non-relevant stuff in your response

You will be provided Input details.<|eot_id|><|start_header_id|>user<|end_header_id|>

Input details:

    Title: The People's Princess, Britons work to keep memory alive

    Title Token Count: 15

    Original Text: LONDON (Reuters) - Abdul Daoud spilt most of the cappuccino into the saucer the first time he serv

In [33]:
print(total_prompts[-1])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 24 Jun 2025

You are an expert summarizer helping a user prepare input for a sentence embedding model with a strict 512-token input limit.
The user will always include the full **title** of the news article. Your job is to **rewrite or summarize the news article text only**, using no more than the available tokens provided. The goal is to preserve **all important meaning** from the article without exceeding the token budget.

You must:
- Leave the title unchanged
- Output **only the rewritten article text**
- Not include anything non-relevant stuff in your response

You will be provided Input details.<|eot_id|><|start_header_id|>user<|end_header_id|>

Input details:

    Title: 'Fully committed' NATO backs new U.S. approach on Afghanistan

    Title Token Count: 16

    Original Text: BRUSSELS (Reuters) - NATO allies on Tuesday welcomed President Donald Trump s decision to com

In [34]:
%%time

outputs = llm.generate(total_prompts, sampling_params)

Processed prompts: 100%|██████████| 19248/19248 [1:24:07<00:00,  3.81it/s, est. speed input: 4486.74 toks/s, output: 822.35 toks/s]    

CPU times: user 1min 18s, sys: 6.48 s, total: 1min 25s
Wall time: 1h 24min 54s


In [35]:
results = [output.outputs[0].text for output in outputs]     

In [36]:
print(results[-1])

NATO allies welcomed President Donald Trump's decision to commit more forces to Afghanistan as part of a new US strategy. The US will send about 4,000 more troops to add to the 8,400 already deployed. Trump's speech did not define benchmarks for ending the war, but he asked allies to support the new strategy with additional troops and funding. NATO Secretary General Jens Stoltenberg said the alliance remains committed to Afghanistan and will discuss the way ahead with Defense Secretary James Mattis and allies. Britain, Germany, and other European allies welcomed the new US plan, citing the need to stabilize the country and reduce the terrorist threat. The US-led invasion of Afghanistan began in 2001, and the country remains weak with a Taliban insurgency nearly 16 years later. Trump said a hasty withdrawal would create a vacuum for groups like Islamic State and al Qaeda to fill.


In [37]:
result_df = pd.DataFrame({"Prompts": total_prompts, "Results": results})

In [38]:
result_df

,Prompts,Results
0,<|begin_of_text|><|start_header_id|>system<|en...,Donald Trump wished Americans a Happy New Year...
1,<|begin_of_text|><|start_header_id|>system<|en...,Former Milwaukee Sheriff David Clarke is facin...
2,<|begin_of_text|><|start_header_id|>system<|en...,Donald Trump announced he'd return to work the...
3,<|begin_of_text|><|start_header_id|>system<|en...,Many people have raised alarm about Donald Tru...
4,<|begin_of_text|><|start_header_id|>system<|en...,"Abigail Disney, an heiress to the Disney empir..."
...,...,...
19243,<|begin_of_text|><|start_header_id|>system<|en...,North Korea and the US clashed at a U.N. forum...
19244,<|begin_of_text|><|start_header_id|>system<|en...,North Korea and the United States clashed at a...
19245,<|begin_of_text|><|start_header_id|>system<|en...,North Korea and the US clashed at the United N...
19246,<|begin_of_text|><|start_header_id|>system<|en...,Two North Korean shipments to a Syrian governm...


In [39]:
result_df.to_csv("llama3_2_3b_summary_results_FinalTesting.csv", index=False)

In [26]:
# import csv
# from tqdm import tqdm

# batch_size = 100
# output_file = "summarized_output.csv"

# # Write headers once
# with open(output_file, "w", newline='', encoding='utf-8') as f:
#     writer = csv.writer(f)
#     writer.writerow(["original_text", "summary"])

In [27]:
# for i in tqdm(range(0, len(total_prompts), batch_size)):
#     batch_prompts = total_prompts[i:i + batch_size]

#     outputs = llm.generate(batch_prompts, sampling_params)

#     summaries = [out.outputs[0].text if out.outputs else "" for out in outputs]

#     with open(output_file, "a", newline='', encoding='utf-8') as f:
#         writer = csv.writer(f)
#         for original, summary in zip(batch_prompts, summaries):
#             writer.writerow([original, summary])